In [155]:
# noexport

import os
os.system('export_notebook browser_libs.ipynb')

-1

In [83]:
import json
import urllib.request as req
from memoize import memoize # pip install memoize2
from pymongo import MongoClient
from getsecret import getsecret
import urllib.parse
import moment
import datetime

In [95]:
#def get_user_to_all_install_ids():
#  user_to_install = json.loads(req.urlopen("http://localhost:5001/get_user_to_all_install_ids").read().decode("utf-8"))
#  return user_to_install

#def get_collection_names():
#  collection_names = json.loads(req.urlopen("http://localhost:5001/listcollections").read().decode("utf-8"))
#  return collection_names

def get_session_info_list_for_user(userid):
  output = json.loads(req.urlopen("http://localhost:5001/get_session_info_list_for_user?userid=" + userid).read().decode("utf-8"))
  return output

In [ ]:
#collection_names = get_collection_items('collections')
#print(len(collection_names))
#print(get_collection_for_user('e0ea34c81d4b50cddc7bd752', 'synced:seconds_on_domain_per_session')[0])

In [84]:
@memoize
def download_url(url):
  return req.urlopen(url).read().decode("utf-8")

def getjson(path, params={}):
  querystring = urllib.parse.urlencode(params)
  url = 'http://localhost:5001/' + path + '?' + querystring
  return json.loads(download_url(url))

def make_getjson_func(path, *param_list):
  def f(*arg_list):
    if len(param_list) != len(arg_list):
      print('missing some number of arguments. expected parameters: ' + str(param_list))
    param_dict = {}
    for param,arg in zip(param_list, arg_list):
      param_dict[param] = arg
    return getjson(path, param_dict)
  return f

def expose_getjson(func_name, *args):
  f = make_getjson_func(func_name, *args)
  globals()[func_name] = f
  return f  


In [85]:
expose_getjson('get_session_info_list_for_user', 'userid')
#expose_getjson('get_user_to_all_install_ids')

#print(get_user_to_all_install_ids()['e0ea34c81d4b50cddc7bd752'])
#get_session_info_list = make_getjson_func('get_session_info_list_for_user', 'userid')
#print(get_session_info_list_for_user('e0ea34c81d4b50cddc7bd752')[0])
#def get_user_to_all_install_ids(user):
#  return getjson

<function __main__.make_getjson_func.<locals>.f(*arg_list)>

In [86]:
@memoize
def get_db(): # this is for the browser
  client = MongoClient(getsecret("EXT_URI"))
  db = client[getsecret("DB_NAME")]
  return db

@memoize
def get_collection_items(collection_name):
  db = get_db()
  return [x for x in db[collection_name].find({})]

def get_collection_for_user(user, collection_name):
  return get_collection_items(user + '_' + collection_name)


In [87]:
def get_collection_names():
  collection_names = get_collection_items('collections')
  return [x['_id'] for x in collection_names]

def get_users_with_goal_frequency_set():
  output = []
  collection_names = get_collection_names()
  for collection_name in collection_names:
    if not collection_name.endswith('_synced:goal_frequencies'):
      continue
    username = collection_name.replace('_synced:goal_frequencies', '')
    output.append(username)
  return output


In [88]:
@memoize
def get_user_to_all_install_ids():
  install_info_list = get_collection_items('installs')
  output = {}
  for install_info in install_info_list:
    if 'user_id' not in install_info:
      continue
    user_id = install_info['user_id']
    install_id = install_info.get('install_id', None)
    if user_id not in output:
      output[user_id] = []
    if install_id not in output[user_id]:
      output[user_id].append(install_id)
  return output

#print(get_user_to_all_install_ids()['e0ea34c81d4b50cddc7bd752'])

In [153]:
@memoize
def get_all_install_ids_for_user(user):
  seconds_on_domain_per_session = get_collection_for_user(user, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_user(user, 'synced:interventions_active_for_domain_and_session')
  user_to_all_install_ids = get_user_to_all_install_ids()
  output = []
  output_set = set()
  if user in user_to_all_install_ids:
    for install_id in user_to_all_install_ids[user]:
      if install_id not in output_set:
        output_set.add(install_id)
        output.append(install_id)
  for item in seconds_on_domain_per_session:
    if 'install_id' not in item:
      continue
    install_id = item['install_id']
    if install_id not in output_set:
      output_set.add(install_id)
      output.append(install_id)
  for item in interventions_active_for_domain_and_session:
    if 'install_id' not in item:
      continue
    install_id = item['install_id']
    if install_id not in output_set:
      output_set.add(install_id)
      output.append(install_id)
  return output

@memoize
def get_is_valid_user(user):
  install_ids = get_all_install_ids_for_user(user)
  if len(install_ids) != 1:
    return False
  return True

@memoize
def get_valid_user_list():
  user_list = get_users_with_goal_frequency_set()
  output = []
  for user in user_list:
    if not get_is_valid_user(user):
      continue
    output.append(user)
  return output

#get_sessions_for_user('e0ea34c81d4b50cddc7bd752')
#valid_user_list = get_valid_user_list()
#print(len(valid_user_list))

In [90]:
'''
function convert_date_to_epoch(date) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  let year = parseInt(date.substr(0, 4))
  let month = parseInt(date.substr(4, 2)) - 1
  let day = parseInt(date.substr(6, 2))
  let date_moment = moment().year(year).month(month).date(day).hours(0).minutes(0).seconds(0).milliseconds(0)
  return date_moment.diff(start_of_epoch, 'days')
}

function convert_epoch_to_date(epoch) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  start_of_epoch.add(epoch, 'days')
  return start_of_epoch.format('YYYYMMDD')
}

function timestamp_to_epoch(timestamp) {
  let start_of_epoch = moment().year(2016).month(0).date(1).hours(0).minutes(0).seconds(0).milliseconds(0)
  return moment(timestamp).diff(start_of_epoch, 'days')
}
'''

def convert_date_to_epoch(date):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  year = int(date[0:4])
  month = int(date[4:6])
  day = int(date[6:8])
  date_moment = moment.now().timezone("US/Pacific").replace(years=year, months=month, days=day, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  return date_moment.diff(start_of_epoch).days

def convert_epoch_to_date(epoch):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch.add(days=epoch)
  return start_of_epoch.format('YYYYMMDD')

def timestamp_to_epoch(timestamp):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  return moment.unix(timestamp).timezone("US/Pacific").diff(start_of_epoch).days

def timestamp_to_isoweek(timestamp):
  isoWeek = int(datetime.datetime.fromtimestamp(timestamp/1000).isocalendar()[1]) 
  return isoWeek

def epoch_to_isoweek(epoch):
  start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch.add(days=epoch)
  timestamp_seconds = start_of_epoch.epoch()
  isoWeek = int(datetime.datetime.fromtimestamp(timestamp_seconds).isocalendar()[1])
  return isoWeek

#print(timestamp_to_epoch(1537059309631))
#print(convert_epoch_to_date(988))
#print(convert_date_to_epoch('20180915'))
#print(convert_date_to_epoch('20180917'))
#print(epoch_to_isoweek(990))

In [57]:
#a=moment.unix(1537221946630)
#dir(a)
#print(timestamp_to_isoweek(1537221946630))

38


In [138]:
@memoize
def get_frequency_info_for_user_epoch(user, epochnum):
  # returns a dictionary mapping goal name -> 1 if frequent, 0 if infrequent
  isoweek_input = epoch_to_isoweek(epochnum)
  goal_frequencies = get_collection_for_user(user, 'synced:goal_frequencies')
  output = {}
  conflict_info_list = []
  for item in goal_frequencies:
    timestamp_local = item['timestamp_local']
    isoweek_local = timestamp_to_isoweek(timestamp_local)
    algorithm_info = json.loads(item['val'])
    algorithm_name = algorithm_info['algorithm']
    onweeks = algorithm_info['onweeks']
    timestamp = algorithm_info['timestamp']
    if algorithm_name == 'isoweek_random':
      is_frequent = onweeks[isoweek_input] == 1
    elif algorithm_name == 'isoweek_alternating':
      is_frequent = isoweek_input % 2 == onweeks
    else:
      raise Exception('unknown frequency selection algorithm ' + algorithm)
    goal = item['key']
    if goal in output:
      conflict_info = {'item': item, 'existing_is_frequent': output[goal], 'is_frequent': is_frequent}
      conflict_info_list.append(conflict_info)
      continue
    output[goal] = is_frequent
    #print(goal)
    #print(is_frequent)
    #print(algorithm_info)
    #print(item)
  return output

def get_is_goal_frequent_for_user_on_domain_at_epoch(user, target_domain, epochnum):
  goal_to_frequency_info = get_frequency_info_for_user_epoch(user, epochnum)
  for goal_name,is_frequent in goal_to_frequency_info.items():
    domain = get_domain_for_goal(goal_name)
    if domain == target_domain:
      return is_frequent
  # we probably shouldn't have gotten here
  return False
  

#def get_frequency_info_for_goal_on_timestamp(user, goal, )

#print(get_frequency_info_for_user_epoch('c11e5f2d93f249b5083989b2', 990))
#print(get_is_goal_frequent_for_user_on_domain_at_epoch('c11e5f2d93f249b5083989b2', 'www.youtube.com', 990))

{'custom/spend_less_time_www.tumblr.com': True, 'reddit/spend_less_time': True, 'youtube/spend_less_time': True, 'custom/spend_less_time_gagadaily.com': False}
True


In [132]:
@memoize
def get_goals_enabled_for_user_sorted_by_timestamp(user):
  goal_info_list = get_collection_for_user(user, 'logs:goals')
  goal_info_list_sorted = []
  for goal_info in goal_info_list:
    if 'timestamp_local' not in goal_info:
      continue
    goal_info_list_sorted.append(goal_info)
  goal_info_list_sorted.sort(key=lambda k: k['timestamp_local'])
  return goal_info_list_sorted

def get_goals_enabled_for_user_at_timestamp(user, target_timestamp_local):
  goal_info_list_sorted = get_goals_enabled_for_user_sorted_by_timestamp(user)
  enabled_goals = {}
  for goal_info in goal_info_list_sorted:
    # note this can be replaced with binary search if it is slow
    timestamp_local = goal_info['timestamp_local']
    if timestamp_local > target_timestamp_local:
      return enabled_goals
    enabled_goals = goal_info['enabled_goals']
  return enabled_goals

def get_is_goal_enabled_for_user_at_timestamp(user, target_goal_name, target_timestamp_local):
  goals_enabled_dictionary = get_goals_enabled_for_user_at_timestamp(user, target_timestamp_local)
  for goal_name,is_enabled in goals_enabled_dictionary.items():
    if goal_name == target_goal_name:
      return is_enabled
  return False

def get_is_goal_enabled_for_user_on_domain_at_timestamp(user, target_domain, target_timestamp_local):
  goals_enabled_dictionary = get_goals_enabled_for_user_at_timestamp(user, target_timestamp_local)
  for goal_name,is_enabled in goals_enabled_dictionary.items():
    domain = get_domain_for_goal(goal_name)
    if domain == target_domain:
      return is_enabled
  return False
    
#print(get_goals_enabled_for_user_sorted_by_timestamp('c11e5f2d93f249b5083989b2')[0])
#print(get_goals_active_for_user_at_timestep('c11e5f2d93f249b5083989b2', 1533450980492.0))

In [115]:
@memoize
def get_goal_intervention_info():
  return json.load(open('goal_intervention_info.json'))

@memoize
def get_goal_info_list():
  goal_intervention_info = get_goal_intervention_info()
  return goal_intervention_info['goals']

@memoize
def get_goal_info_dict():
  goal_info_list = get_goal_info_list()
  output = {}
  for goal_info in goal_info_list:
    goal_name = goal_info['name']
    output[goal_name] = goal_info
  return output

@memoize
def get_domain_for_goal(goal_name):
  goal_info_dict = get_goal_info_dict()
  if goal_name in goal_info_dict:
    return goal_info_dict[goal_name]['domain']
  if goal_name.startswith('custom/spend_less_time_'): # custom/spend_less_time_www.tumblr.com
    return goal_name[23:] # 23 == len('custom/spend_less_time_www.tumblr.com')
  raise Exception('could not find domain for goal ' + goal_name)

#get_goal_info_dict()
#print(get_domain_for_goal('youtube/spend_less_time'))
#print(get_domain_for_goal('custom/spend_less_time_www.tumblr.com'))

In [147]:
def get_sessions_for_user(user):
  seconds_on_domain_per_session = get_collection_for_user(user, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_user(user, 'synced:interventions_active_for_domain_and_session')
  #print(seconds_on_domain_per_session[0])
  #print(interventions_active_for_domain_and_session[0])
  output = []
  domain_to_session_id_to_duration_info = {}
  domain_to_session_id_to_intervention_info = {}
  interventions_deployed_with_no_duration_info = []
  seconds_on_domain_per_session.sort(key=lambda k: k['timestamp_local'])
  for item in seconds_on_domain_per_session:
    domain = item['key']
    session_id = item['key2']
    if domain not in domain_to_session_id_to_duration_info:
      domain_to_session_id_to_duration_info[domain] = {}
    domain_to_session_id_to_duration_info[domain][session_id] = item
  for item in interventions_active_for_domain_and_session:
    domain = item['key']
    session_id = item['key2']
    if domain not in domain_to_session_id_to_intervention_info:
      domain_to_session_id_to_intervention_info[domain] = {}
    domain_to_session_id_to_intervention_info[domain][session_id] = item
  for item in seconds_on_domain_per_session:
    #print(item)
    domain = item['key']
    duration = item['val']
    is_goal = False
    local_timestamp = item['timestamp_local']
    local_epoch = timestamp_to_epoch(local_timestamp)
    interventions_active_info = None
    interventions_active_list = None
    intervention_active = None
    if (domain in domain_to_session_id_to_intervention_info) and (session_id in domain_to_session_id_to_intervention_info[domain]):
      interventions_active_info = domain_to_session_id_to_intervention_info[domain][session_id]
      interventions_active_list = json.loads(interventions_active_info['val'])
      if len(interventions_active_list) > 0:
        intervention_active = interventions_active_list[0]
    goals_enabled = get_goals_enabled_for_user_at_timestamp(user, local_timestamp)
    is_goal_enabled = get_is_goal_enabled_for_user_on_domain_at_timestamp(user, domain, local_timestamp)
    is_goal_frequent = get_is_goal_frequent_for_user_on_domain_at_epoch(user, domain, local_epoch)
    goal_to_frequency_info = get_frequency_info_for_user_epoch(user, local_epoch)
    output.append({
      'domain': domain,
      'is_goal_enabled': is_goal_enabled,
      'is_goal_frequent': is_goal_frequent,
      'intervention_active': intervention_active,
      'duration': duration,
      'local_timestamp': local_timestamp,
      'local_epoch': local_epoch,
    })
    #if interventions_active_info != None and interventions_active_list != None and len(interventions_active_list) > 0:
    #  print(domain)
    #  print(is_goal_enabled)
    #  print(intervention_active)
    #  print(duration)
    #  print(is_goal_frequent)
    #  print(goals_enabled)
    #  print(goal_to_frequency_info)
    #  return
  #  duration = item['val']
  #  print(duration)
  return output

#print(get_sessions_for_user('c11e5f2d93f249b5083989b2'))

In [154]:
#all_sessions_info_list = []
#for user in get_valid_user_list():
#  print(user)
#  for info in get_sessions_for_user(user):
#    info['user'] = user
#    all_sessions_info_list.append(info)
    

c11e5f2d93f249b5083989b2
5d73eb22468ea27cdfdd5e2a
2b5972416d3df31ad7b1c25f
46f96a6ab8753a767bdd1b1b
30134f4f9d24141ae0bca891
11f367d7c4c9743aabbda7ff
e6be8edae93ade10b9c7f78d
d45023abf8d0638072dfce0e
14fd34ba7978f5c31557000d
b96c4f7eef2ab6d081013d3b
4af81d6f6b903b27f34c5f41
921b2a944cc8faa54efa7e89
8ad445f80101c40301f1ca52
9514fe891b9ee2a083cb7cb6
3aa9e9b4f04c7f4d5a61b58e
dd6e625a532e9dda1fcfc08e
44f163249790127e475a4d22
17e7de4f422a37ce6b911178
68ce22b94204c4959b3bca47
b2f7cba6e365326b0867adaa
5655fa9ee5d384a75feacd10
fd208d6ea1f97ac6e8d3c647
790019388193eb88b81aca56
30f66edc6f4522a2466d5ce0
6b719706061bd70657d8908a
befa7276e957100eebe6bf2c
4695deda6b34b39075b78589
c8b0e3c4f63d4dc18538d44f
8aa90cc3b2a3d1318b82999a
d0653b1b154105cae6b20f5a
ed86421a4167fd4d84b5dc4a
8e445b95a635466551c7fe9a
7de231e89272243eef3728c9
d56a186f73ff9c6f157e22f6
413790b746780d4712adad4b
31a2b705e857b148ca832ca6
c6424c6fc44ce87b7313e4ea
f6ac752bff889550a3b3f023
eba693ff2cc67e6a1530697d
12d99769ecc3de0f63b3352d


76f531e30e91efbc36c14e54
52d53bf09504aa2809bf89db
4a1b42cec9b3f3c6a28dec79
b93c29d71bfa7c0c65c27ed6
3e9a053e213799292530dc45
b9c966e4ab689d7f43b1c7c6
daa3c6056c885691fd914e1f
0b21ec3a24f6fa2fc39fa3e4
aa7c9dbe3f39128bd7338aad
dda69f53e39a7911f04b968f
7cc1734a7dcd53e8bde19644
3b1f9b5a274f17ccf5264212
f727a9429bba6605cb61aea6
e777f749ea5d954e56f34a4d
4befb3b05c62bccf640f3111
c610b861496efaffdc84b916
5efd6a1afb4dc00ddfcfbf5e
940b4fadfffbd8485b065712
18d5f3a11194e06a1a802d90
c17d9fe54897d2810651945f
00aa8c2cee6715496a16d16f
12706d74234cc86fb0bdf389
98dfdbbc3eab8a7b09d82fe6
80eceaab0ac4fb7a7f704de1
dcb0e1e6a3b82ceb4df57946
44cf2a5b8eb27257e95ffff2
93c9adf38f6420c946bc573e
8d9107f488014b83c2373a44
4160c051e4efec834618763e
4e4c681a91ac0bedfa8a79bb
2d079677b02a127cc1024f45
fecd855dea0b5cbbbbced63f
83ada9b334ddc4f28251c3b2
ea73737cd3de56d4731c3772
9221d6d695525d3e4381fb43
1760db59ff53fff8c4b58d03
b007a45d8934198b1a163624
dceace4d4bea20a723395f73
7cfd4e734773f412f172717b
ce44110efa8ec51d7602fad5


ca55f0a0e2e0765089529619
ade96c4b5e35cebc2fb28c8d
4fa9cf944d268f312fbddde0
d936520e1e4bffc3b0c90a00
6ca2b25832e5b632fb7c8a0e
88fbf124262ff9b7283faa08
e4c63ee76fed079f0101a92f
87910c588c073c3781dd6ebc
a3cf5d18447d24cb05b18728
ca66a27ca710996cfe982a8f
aaf3508a72648634b1744146
8e41fc28764f68e382f71046
0d7454c89518fae455c1b51a
ef2c8197a97dc46cc42b4b46
f7785f14dc3df84a6c74b373
6730989af247dc509b6a793e
2e9615a29018c6a73f3de0cb
47244e9ac86169fc8686486c
7a988bbeb7efa657592eb0bd
d9a00feddf51b76fd422f8e3
5d81741a5efbef080c2402eb
60865fd2149d691aedb62333
99fdf78984d9505fd0619d9a
ae016d3002632bb1ec08f8ad
c2b63cb8485c1620aafb93bd
b20032a912d904987aff2035
b70db797834905c52f99e542
51c8f02229b79aec8f60d900
a0b1070c92f1a2ac42db5295
f406d9584468e71f7fa8b0d1
efc5af7fe6b85417e66b6b2b
b2f3bec5ce2d2a653d575015
5b1145f3c6a209dd812b6886
a8aad9b957736ecd19f5ede6
488ef3d20a236120bcbc7ad7
d2bba715c749af98e670c602
b6c7c2899fab55287a07f4a8
5202f097aeddd58a30436262
36490019731b04d996285c2f
895fd15d68289a41712d4c19


a9bc8ff89785d653605e67bd
e48d722331177b07aae18cfb
97818beb508a507e92cab0bb
2041fd3be167bc8c9b6f56aa
fa79c3f141adf084662da2bb
8d5c45b7c24ab81decb8b6ce
b11fed17be245935a82b2329
635d8fa1dfabc9821f81e54c
2eb6282b7f54c4f8a830077d
2f74e41d9987831a22dc6900
586f60f78a9a9f3ec962b8d3
c63f331f99db69839b782fab
226621ce4335e4415c72fab0
cae27c94ba09ef0ab74205dd
4ef3dc2416986ba1626e2a3a
fb234ed7d2771053374550f4
15d135f93ee2ddae69df6f14
3935d43de63d6e39834b7e1d
0a8d845b95ccc055c778895d
0cc3e5a34dcb12156268219f
339cd5bc522749ca10e089b0
18df23ff91cef5e9ef33ee06
9c3a7c014d7c80ad2fde6787
61fa6550a4bbcc6df9b3834a
1e37a6bb023f3b28a247b860
d079008d1d9a5201e2043770
c4649581843fb912e3fe73a6
ece9aa9684880ee98f6c5e76
8f2201603c3aaa33d1b25873
a00d3cbf5abb704aab5892c8
2085b9d99b3ee8220506d9f7
14558ecbf176a567a46079cb
61467cb0699a3378ece71e00
50031a6fa01002604513aec5
5622a918b4ff4b29a1f22d63
746b5aca7f41eb2b49cb1baf
09e39eceb7f4620567e9041d
4e8fd092a6f3d576b60ef079
adfbcb584bb1bb19604a5200
b424be3cef5a669a66d807d1


In [149]:
def group_sessions_by_domain(session_info_list):
  output = {}
  for item in session_info_list:
    domain = item['domain']
    if domain not in output:
      output[domain] = []
    output[domain].append(item)
  return output

def group_sessions_by_epoch(session_info_list):
  output = {}
  for item in session_info_list:
    epoch = item['local_epoch']
    if epoch not in output:
      output[epoch] = []
    output[epoch].append(item)
  return output

def get_sessions_for_user_by_day_and_goal(user):
  session_info_list = get_sessions_for_user(user)
  for epoch,session_info_list_for_day in group_sessions_by_epoch(session_info_list).items():
    for domain,session_info_list_for_domain in group_sessions_by_domain(session_info_list_for_day).items():
      other_goal_domain_total_time = 0
      
      print(epoch)
      print(domain)
      print(session_info_list)
      return
  return

get_sessions_for_user_by_day_and_goal('c11e5f2d93f249b5083989b2')

946
www.tumblr.com
[{'domain': 'www.tumblr.com', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 17, 'local_timestamp': 1533451014441.0, 'local_epoch': 946}, {'domain': 'gagadaily.com', 'is_goal_enabled': False, 'is_goal_frequent': True, 'intervention_active': None, 'duration': 3, 'local_timestamp': 1533451098481.0, 'local_epoch': 946}, {'domain': 'www.tomsguide.com', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 611, 'local_timestamp': 1533451317472.0, 'local_epoch': 946}, {'domain': 'downloads.tomsguide.com', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 10, 'local_timestamp': 1533451358452.0, 'local_epoch': 946}, {'domain': 'chrome.google.com', 'is_goal_enabled': False, 'is_goal_frequent': False, 'intervention_active': None, 'duration': 84, 'local_timestamp': 1533451369451.0, 'local_epoch': 946}, {'domain': 'downloads.tomsguide.com', 'is_goal_

In [ ]:
# def get_days_and_sessions_for_user(user):
#   session_info_list = get_sessions_for_user(user)
#   min_epoch = min([x['local_epoch'] for x in session_info_list])
#   max_epoch = max([x['local'] for x in session_info_list])
#   for epoch in range(min_epoch, max_epoch + 1):
    

# print(get_days_and_sessions_for_user('c11e5f2d93f249b5083989b2'))

In [20]:
#print(len(get_users_with_goal_frequency_set()))
#print(valid_user_list[0])

In [ ]:
def print_stats_on_install_records():
  user_to_all_install_ids = get_user_to_all_install_ids()
  users_with_goal_frequency_set = get_users_with_goal_frequency_set()
  users_with_missing_install_record = []
  users_with_zero_installs = []
  users_with_multiple_installs = []
  users_with_single_install = []

  for username in users_with_goal_frequency_set:
    if username not in user_to_all_install_ids:
      users_with_missing_install_record.append(username)
      continue
    install_ids = user_to_all_install_ids[username]
    if len(install_ids) == 0:
      users_with_zero_installs.append(username)
      continue
    if len(install_ids) > 1:
      users_with_multiple_installs.append(username)
      continue
    users_with_single_install.append(username)

  print('users with missing install record', len(users_with_missing_install_record))
  print('users with zero installs', len(users_with_zero_installs))
  print('users with multiple installs', len(users_with_multiple_installs))
  print('users with single install', len(users_with_single_install))


#print_stats_on_install_records()